In [31]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from skimage.io import imread
import numpy as np
import cv2 as cv
import pickle
import os

In [2]:
def loadImg(path, img):
    """Load img from imgs folder

    Args:
        path (str): path of file

    Returns:
        im: image in grayscale
    """
    # color
    # im = cv.imread(os.path.join(path, img), cv.IMREAD_COLOR)
    # im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
    im = imread(os.path.join(path, img))

    return im


X = total samples shape (total , tamaño HOG)
y = np.append (np.ones(shpae=))

In [3]:
def getHog(folderP,folderB):
    """ get HOG of all imgs

    Args:
        folderP (_type_): _description_

    Returns:
        _type_: _description_
    """
    lenFP = len(os.listdir(folderP))
    lenFB = len(os.listdir(folderB))
    total = lenFP+lenFB
    # print(total)
    X = np.zeros(shape=(total,3780))
    y = np.append(np.ones(shape=(1,lenFP)),np.zeros(shape=(1,lenFB)))
    count = 0
    for filename in os.listdir(folderP):
        img = loadImg(folderP,filename)
        if img is not None:
            hogImg = hog(img,orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
            block_norm='L2', feature_vector=True, channel_axis=2) # feature vector= true, channel_Axis=2
            X[count] = hogImg
        count += 1

    for filename in os.listdir(folderB):
        img = loadImg(folderB,filename)
        if img is not None:
            hogImg = hog(img,orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
            block_norm='L2', feature_vector=True, channel_axis=2) # feature vector= true, channel_Axis=2
            X[count] = hogImg
        count += 1
    return X,y

# Cargar data 
- Se toma 53s con el dataset completo

In [4]:
# folderP = 'Pedestrians-Dataset-complete/Pedestrians-Dataset-Dummy/Pedestrians'
# folderB = 'Pedestrians-Dataset-complete/Pedestrians-Dataset-Dummy/Background'
folderP = 'Pedestrians-Dataset-complete/Pedestrians-Dataset/Pedestrians'
folderB = 'Pedestrians-Dataset-complete/Pedestrians-Dataset/Background'
X, y = getHog(folderP, folderB)


# Separar data
- X & y no son del mismo tamaño!!! ⚠️
- Botamos el extra?

In [5]:
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Adaboost

In [38]:
def model(X,y, modelType="ABC"):
    # Create adaboost classifer object
    if modelType == "ABC":
        mod = AdaBoostClassifier(n_estimators=150, learning_rate=1)
    elif modelType == "GBC":
        mod = GradientBoostingClassifier(n_estimators=150, learning_rate=1)
    # Train Adaboost Classifer
    model = mod.fit(X, y)
    
    return model


In [39]:
#Predict the response for test dataset
modelTrained = model(xTrain,yTrain, "GBC")

In [40]:
yPred = modelTrained.predict(xTest)

In [19]:
precision_score(yTest, yPred)

0.9366576819407008

In [41]:
precision_score(yTest, yPred)

0.9271255060728745

In [42]:
recall_score(yTest, yPred)

0.9003931847968545

In [ ]:
yTest

## Pickle
<img src="Pickle_rick_transparent_edgetrimmed.webp" style="height: 160px; width:120px;"/>


In [ ]:
def saveModel(model, path=""):
    with open(path+'modelPkl', 'wb') as files:
        pickle.dump(model, files)

def loadModel(path=""):
    with open(path+'modelPkl' , 'rb') as files:
        model = pickle.load(files)
    return model
